In [33]:
import torch
from abc import ABC, abstractmethod

class BaseBayesianOptimization(ABC):
    def __init__(self, bounds):
        self.bounds = bounds
        self.train_X = None
        self.train_Y = None
        self.model = None

    @abstractmethod
    def initialize(self, X_init, Y_init):
        pass

    @abstractmethod
    def _fit_model(self):
        pass

    @abstractmethod
    def acquisition_function(self):
        pass

    @abstractmethod
    def optimize_acquisition(self, acq_function):
        pass

    @abstractmethod
    def step(self):
        pass

    @abstractmethod
    def update(self, new_X, new_Y):
        pass

class BayesianOptimization(BaseBayesianOptimization):
    def __init__(self, bounds, beta=2.0):
        super().__init__(bounds)
        self.beta = beta

    def initialize(self, X_init, Y_init):
        self.train_X = X_init
        self.train_Y = Y_init
        self.model = self._fit_model()

    def _fit_model(self):
        from botorch.models import SingleTaskGP
        from gpytorch.mlls import ExactMarginalLogLikelihood
        from botorch.fit import fit_gpytorch_mll
        
        model = SingleTaskGP(self.train_X, self.train_Y)
        mll = ExactMarginalLogLikelihood(model.likelihood, model)
        fit_gpytorch_mll(mll)
        return model
    
    def acquisition_function(self):
        from botorch.acquisition import UpperConfidenceBound
        
        UCB = UpperConfidenceBound(self.model, beta=self.beta)
        return UCB

    def optimize_acquisition(self, acq_function):
        from botorch.optim import optimize_acqf
        
        candidates, _ = optimize_acqf(
            acq_function,
            bounds=self.bounds,
            q=1,
            num_restarts=10,
            raw_samples=20,
        )
        return candidates

    def step(self):
        acq_function = self.acquisition_function()
        new_X = self.optimize_acquisition(acq_function)
        return new_X

    def update(self, new_X, new_Y):
        self.train_X = torch.cat([self.train_X, new_X], dim=0)
        self.train_Y = torch.cat([self.train_Y, new_Y], dim=0)
        self.model = self._fit_model()

# Example Usage
if __name__ == "__main__":
    bounds = torch.tensor([[0.0], [1.0]])
    X_init = torch.rand(5, 1)
    Y_init = torch.sin(X_init * (2 * torch.pi))

    bo = BayesianOptimization(bounds=bounds)
    bo.initialize(X_init, Y_init)

    for i in range(10):
        new_X = bo.step()
        new_Y = torch.sin(new_X * (2 * torch.pi))
        bo.update(new_X, new_Y)
        print(f"Iteration {i+1}: Suggested point: {new_X.numpy()}, Function value: {new_Y.numpy()}")

/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/895626148.py:50: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  model = SingleTaskGP(self.train_X, self.train_Y)
/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning: Data is not standardized (std = tensor([0.7541]), mean = tensor([-0.1095])). Please consider scaling the input to zero mean and unit variance.
  check_standardization(Y=train_Y, raise_on_fail=raise_on_fail)
/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/895626148.py:50: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help 

Iteration 1: Suggested point: [[0.]], Function value: [[0.]]
Iteration 2: Suggested point: [[0.23685773]], Function value: [[0.9965926]]
Iteration 3: Suggested point: [[0.2712205]], Function value: [[0.9911244]]


/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/895626148.py:50: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  model = SingleTaskGP(self.train_X, self.train_Y)
/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning: Data is not standardized (std = tensor([0.7415]), mean = tensor([0.0641])). Please consider scaling the input to zero mean and unit variance.
  check_standardization(Y=train_Y, raise_on_fail=raise_on_fail)
/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/895626148.py:50: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help a

Iteration 4: Suggested point: [[0.2510879]], Function value: [[0.99997663]]
Iteration 5: Suggested point: [[0.2524308]], Function value: [[0.99988335]]
Iteration 6: Suggested point: [[0.24982424]], Function value: [[0.9999994]]
Iteration 7: Suggested point: [[0.2521402]], Function value: [[0.9999096]]


/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/895626148.py:50: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  model = SingleTaskGP(self.train_X, self.train_Y)
/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning: Data is not standardized (std = tensor([0.7547]), mean = tensor([0.3440])). Please consider scaling the input to zero mean and unit variance.
  check_standardization(Y=train_Y, raise_on_fail=raise_on_fail)
/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/895626148.py:50: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help a

Iteration 8: Suggested point: [[0.2412628]], Function value: [[0.9984935]]
Iteration 9: Suggested point: [[0.25649184]], Function value: [[0.9991682]]
Iteration 10: Suggested point: [[0.2513559]], Function value: [[0.9999637]]


/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/895626148.py:50: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  model = SingleTaskGP(self.train_X, self.train_Y)
/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning: Data is not standardized (std = tensor([0.6991]), mean = tensor([0.5312])). Please consider scaling the input to zero mean and unit variance.
  check_standardization(Y=train_Y, raise_on_fail=raise_on_fail)
/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/895626148.py:50: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help a

In [38]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf
from gpytorch.mlls import ExactMarginalLogLikelihood

class BaseBayesianOptimization(ABC):
    def __init__(self, bounds):
        self.bounds = bounds
        self.train_X = None
        self.train_Y = None
        self.model = None

    @abstractmethod
    def initialize(self, X_init, Y_init):
        pass

    @abstractmethod
    def _fit_model(self):
        pass

    @abstractmethod
    def acquisition_function(self):
        pass

    @abstractmethod
    def optimize_acquisition(self, acq_function):
        pass

    @abstractmethod
    def step(self):
        pass

    @abstractmethod
    def update(self, new_X, new_Y):
        pass

class DiscreteBO(BaseBayesianOptimization):
    def __init__(self, bounds, beta=2.0, beta_h=10.0, l_h=2.0):
        super().__init__(bounds)
        self.beta = beta
        self.beta_h = beta_h
        self.l_h = l_h
        self.l = 1.0

    def initialize(self, X_init, Y_init):
        self.train_X = X_init
        self.train_Y = Y_init
        self.model = self._fit_model()

    def _fit_model(self):
        model = SingleTaskGP(self.train_X, self.train_Y)
        mll = ExactMarginalLogLikelihood(model.likelihood, model)
        fit_gpytorch_mll(mll)
        return model

    def acquisition_function(self, beta):
        return UpperConfidenceBound(self.model, beta=beta)

    def optimize_acquisition(self, acq_function):
        candidates, _ = optimize_acqf(
            acq_function,
            bounds=self.bounds,
            q=1,
            num_restarts=10,
            raw_samples=20,
        )
        return candidates

    def step(self):
        acq_function = self.acquisition_function(self.beta)
        new_X = self.optimize_acquisition(acq_function)
        return new_X

    def update(self, new_X, new_Y):
        self.train_X = torch.cat([self.train_X, new_X], dim=0)
        self.train_Y = torch.cat([self.train_Y, new_Y], dim=0)
        self.model = self._fit_model()

    def adjust_beta_and_l(self):
        # Define the optimization problem to adjust beta and l
        def objective(params):
            delta_beta, l = params
            adjusted_beta = self.beta + delta_beta
            acq_function = self.acquisition_function(adjusted_beta)
            new_X = self.optimize_acquisition(acq_function)
            rounded_new_X = torch.round(new_X)

            penalty = float('inf')
            if (rounded_new_X == self.train_X).all(dim=1).any():
                penalty = 1000  # Arbitrary high value to avoid repetition

            return delta_beta + torch.norm(new_X - rounded_new_X).item() + penalty

        # Initial guess and bounds for delta_beta and l
        initial_guess = torch.tensor([0.0, self.l])
        bounds = [(0.0, self.beta_h), (1e-3, self.l_h)]

        # Optimize the objective function
        result = torch.minimize(objective, initial_guess, bounds=bounds, method='L-BFGS-B')
        delta_beta, l = result.x
        self.beta += delta_beta
        self.l = l

        print()
        print()
        print()
        print()
        print()
        print()
        print()

# Example Usage
if __name__ == "__main__":
    bounds = torch.tensor([[0.0], [1.0]])
    X_init = torch.rand(5, 1)
    Y_init = torch.sin(X_init * (2 * torch.pi))

    bo = DiscreteBO(bounds=bounds)
    bo.initialize(X_init, Y_init)

    for i in range(10):
        new_X = bo.step()
        if (new_X == bo.train_X).all(dim=1).any():
            bo.adjust_beta_and_l()
            new_X = bo.step()
        new_Y = torch.sin(new_X * (2 * torch.pi))
        bo.update(new_X, new_Y)
        print(f"Iteration {i+1}: Suggested point: {new_X.numpy()}, Function value: {new_Y.numpy()}")

/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/3167925398.py:53: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  model = SingleTaskGP(self.train_X, self.train_Y)
/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning: Data is not standardized (std = tensor([0.8351]), mean = tensor([0.4630])). Please consider scaling the input to zero mean and unit variance.
  check_standardization(Y=train_Y, raise_on_fail=raise_on_fail)
/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/3167925398.py:53: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help

Iteration 1: Suggested point: [[0.]], Function value: [[0.]]
Iteration 2: Suggested point: [[0.22848748]], Function value: [[0.9908788]]
Iteration 3: Suggested point: [[0.2544213]], Function value: [[0.9996142]]
Iteration 4: Suggested point: [[0.25276524]], Function value: [[0.9998491]]


/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/optim/optimize.py:564: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.')]
Trying again with a new set of initial conditions.
  return _optimize_acqf_batch(opt_inputs=opt_inputs)
/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/3167925398.py:53: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  model = SingleTaskGP(self.train_X, self.train_Y)
/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning: Data is not standardized (std = te

Iteration 5: Suggested point: [[0.2524194]], Function value: [[0.9998845]]
Iteration 6: Suggested point: [[0.25098068]], Function value: [[0.99998105]]
Iteration 7: Suggested point: [[0.25091198]], Function value: [[0.9999836]]


/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/3167925398.py:53: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  model = SingleTaskGP(self.train_X, self.train_Y)
/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning: Data is not standardized (std = tensor([0.6101]), mean = tensor([0.6921])). Please consider scaling the input to zero mean and unit variance.
  check_standardization(Y=train_Y, raise_on_fail=raise_on_fail)
/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/3167925398.py:53: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help

Iteration 8: Suggested point: [[0.25210455]], Function value: [[0.99991256]]
Iteration 9: Suggested point: [[0.2525134]], Function value: [[0.9998753]]
Iteration 10: Suggested point: [[0.251264]], Function value: [[0.99996847]]


/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/3167925398.py:53: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  model = SingleTaskGP(self.train_X, self.train_Y)
/Users/keisukeonoue/ws/constrained_BO/bo-env/lib/python3.12/site-packages/botorch/models/utils/assorted.py:260: InputDataWarning: Data is not standardized (std = tensor([0.5722]), mean = tensor([0.7361])). Please consider scaling the input to zero mean and unit variance.
  check_standardization(Y=train_Y, raise_on_fail=raise_on_fail)
/var/folders/_b/b62kq0490fn2ptq7m4mbp8yr0000gp/T/ipykernel_39684/3167925398.py:53: InputDataWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help